# Avalanche chains exploration (X-chain, C-chain)

## Imports and constants

In [ ]:
import requests
import json
import logging
import sys
import pandas as pd
import unittest
import time
from json.decoder import JSONDecodeError
import black
import jupyter_black

c_chain_api = 'https://api.avax.network/ext/bc/C/rpc'
x_chain_api = 'https://api.avax.network/ext/bc/X'
headers = {'content-type': 'application/json'}
covalent_api_key = "?key=ckey_2945523d713f4b50acc58fdb7b9"
chain_id = '43114/'
covalent_endpoint = 'https://api.covalenthq.com/v1/'
avax_decim = 10**18
pd.options.display.float_format = '{:,.4f}'.format



## Logger and formatter

In [ ]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
formatter = logging.Formatter(
    "%(asctime)s - %(name)s - %(funcName)s - %(levelname)s - %(message)s"
)
consoleHandler = logging.StreamHandler(sys.stdout)
consoleHandler.setLevel(logging.ERROR)
consoleHandler.setFormatter(formatter)
logger.addHandler(consoleHandler)
logging.getLogger("urllib3").setLevel(logging.ERROR)
logging.getLogger("blib2to3").setLevel(logging.ERROR)
logger.propagate = False

# code formatter
jupyter_black.load()

## Functions for getting and outputting data from C-chain and X-chain public APIs

In [ ]:
def aval_response_error_logger(response):
    """Function for outputting error logs if no result is fetched from AVAX Public APIs

    Params
    ----------
    response: response
        API response
    """

    logger.error("Error message: {}".format(response.json()["error"]["message"]))


def make_avalanche_request(chain, method, params):
    """Function for creating calls to AVAX Public APIs, works for both chains (X,C)

     Params
     ----------
    chain: str
        string with endpoint for X-chain or C-chain
    method: str
        string with method name to be called from API
    params: dict or list, depending on chain
        request parameters

     Returns
     ----------
     response: response
         response from API
    """
    response = requests.post(
        chain,
        headers=headers,
        data=json.dumps(
            {"jsonrpc": "2.0", "id": 1, "method": method, "params": params}
        ),
    )
    return response


def get_avalanche_c_balance(addresses):  # C-chain addresses AVAX
    """Function for getting info on wallets' balances from C-chain AVAX Public API,
       !!!  supports only Avalanche Native Tokens, no ARC-20, etc.

     Params
     ----------
    addresses: list
        list with C-chain addresses

     Returns
     ----------
     res: dict
         dict with balances information gathered from C-chain API

    """
    res = {}
    if not isinstance(addresses, list):
        logger.error("Invalid input type. List is required")
        return res
    for a in addresses:
        response = make_avalanche_request(c_chain_api, "eth_getBalance", [a, "latest"])
        try:
            balance = response.json()["result"]
            res[a] = int(balance, 16) / avax_decim
        except KeyError:
            aval_response_error_logger(response)
    return res


def get_avalanche_c_asset_balance(addresses, asset_id):
    """Function for getting info on wallets' balances of particular asset from C-chain AVAX Public API,
       !!!  supports only Avalanche Native Tokens, no ARC-20, etc.

     Params
     ----------
    addresses: list
        list with C-chain addresses

     asset_id: str
         X-chain asset id

     Returns
     ----------
     res: dict
         dict with balances information gathered from C-chain API

    """
    res = {}
    if not isinstance(addresses, list):
        logger.error("Invalid input type. List is required")
        return res
    for a in addresses:
        response = make_avalanche_request(
            c_chain_api, "eth_getAssetBalance", [a, "latest", asset_id]
        )
        try:
            balance = response.json()["result"]
            res[a] = int(balance, 16)
        except KeyError:
            aval_response_error_logger(response)
    return res


def get_avalanche_x_balances(addresses):  # X-chain addresses
    """
     Function for getting info on wallets' balances from X-chain AVAX Public API

     Params
     ----------
    addresses: list
        list with X-chain addresses

     Returns
     ----------
     res: dict
         dict with balances information gathered from X-chain API
    """
    res = {}
    if not isinstance(addresses, list):
        logger.error("Invalid input type. List is required")
        return res
    for a in addresses:
        response = make_avalanche_request(
            x_chain_api, "avm.getAllBalances", {"address": a}
        )
        try:
            balances = response.json()["result"]["balances"]
            res[a] = balances
        except KeyError:
            aval_response_error_logger(response)

    return res


def get_asset_description(assetid):  # ANT ids
    """
     Function for getting info on Avalanche Native Token type asset from X-chain API

     Params
     ----------
    assetid: str
        asset id (e.g. kV2znPTurNPJB7PxXbpDQBWDSifzspcp21hy7HbBVMzVVboy9)

     Returns
     ----------
     desc: dict
         dict with information on asset gathered from X-chain API
    """
    desc = {}
    if not isinstance(assetid, str):
        logger.error("Invalid input type. String is required")
        return desc
    response = make_avalanche_request(
        x_chain_api, "avm.getAssetDescription", {"assetID": assetid}
    )
    try:
        desc = response.json()["result"]
    except KeyError:
        aval_response_error_logger(response)
    return desc


def output_c_balances(balances):
    """
     Function for generating DataFrame out of C-chain balances info

     Params
     ----------
    balances: dict
        C-chain balances information

     Returns
     ----------
     df: pd.DataFrame
         Same information in DataFrame format
    """
    if not isinstance(balances, dict):
        logger.error("Invalid input type. Dict is required. Returning None")
        return None
    elif balances == {}:
        logger.error("Empty dict is provided for df formation. Returning None")
        return None

    d_df = {"address": [], "balance": []}
    for k, v in balances.items():
        d_df["address"].append(k)
        d_df["balance"].append(v)
    df = pd.DataFrame.from_dict(d_df, orient="columns")
    return df


def output_x_balances(balances):
    """
     Function for generating DataFrame out of X-chain balances info

     Params
     ----------
    balances: dict
        X-chain balances information

     Returns
     ----------
     df: pd.DataFrame
         Same information in DataFrame format
    """
    if not isinstance(balances, dict):
        logger.error("Invalid input type. Dict is required. Returning None")
        return None
    elif balances == {}:
        logger.error("Empty dict is provided for df formation. Returning None")
        return None
    d_df = {"address": [], "asset_name": [], "balance": [], "asset": []}
    for add, blns in balances.items():
        if blns == []:
            d_df["address"].append(add)
            d_df["asset_name"].append(None)
            d_df["balance"].append(None)
            d_df["asset"].append(None)
        else:
            for b in blns:
                d_df["address"].append(add)
                asset_desc = get_asset_description(b["asset"])
                d_df["asset_name"].append(
                    asset_desc["symbol"] + ": " + asset_desc["name"]
                )
                d_df["balance"].append(
                    int(b["balance"]) / 10 ** int(asset_desc["denomination"])
                )
                d_df["asset"].append(b["asset"])
    df = pd.DataFrame.from_dict(d_df, orient="columns")
    return df

### Insert X-chain addresses to the list to output balances
#### Addresses with empty balances will be outputted as None/NaN 
#### Assets on X-chain are AVAX and ANTs (Avalanche Native Tokens) 

In [ ]:
x_chain_addresses = [
    "X-avax18jqvtngyu2ahdkfrkmyl059unh9hagjeq6m7gu",
    "X-avax1slt2dhfu6a6qezcn5sgtagumq8ag8we75f84sw" # some address with ANTs
]  
x_balances = get_avalanche_x_balances(x_chain_addresses)
output_x_balances(x_balances)

### Getting C-chain balances from the chain public API (AVAX public API method returns AVAX balances only)

### Insert C-chain addresses in addresses list

In [ ]:
addresses = [
    "0x1d7dCCBf623584009BDE3229724D29Ae5AC09d1f",
    "0x3808FD461D246D6F7BD0a02fb6bC3a77AD8922Dd",
    "0x2B1855f3E0d453faF76c91EA8504239D3bbEDb61",
    "0x0bE48D32539966c243C969ca566b7b750CF4C30b",
    "0x66f59CD59DaE5cf5d42832af087d89eC7Aaf4D74",
]
c_balances = get_avalanche_c_balance(addresses)  # Avax
output_c_balances(c_balances)

### Getting some ANT balance from C-chain public API based on asset id

### Insert C-chain addresses in addresses list

In [ ]:
addresses = [
    "0x1d7dCCBf623584009BDE3229724D29Ae5AC09d1f",
    "0x3808FD461D246D6F7BD0a02fb6bC3a77AD8922Dd",
    "0x2B1855f3E0d453faF76c91EA8504239D3bbEDb61",
    "0x0bE48D32539966c243C969ca566b7b750CF4C30b",
    "0x66f59CD59DaE5cf5d42832af087d89eC7Aaf4D74",
]
balances = get_avalanche_c_asset_balance(
    addresses, "TmZy46qkZtgCi7Xv9sgZC41XjrLp5L6PDpqwRiPw6b9aidhTJ" # some nft
)  
output_c_balances(balances)

## Getting C-chain Wallet balances with Covalent API

In [ ]:
def coval_response_error_logger(response):
    """Function for outputting error logs if no result is fetched from AVAX Public APIs

    Params
    ----------
    response: json
        API response in json format
    """
    logger.error(
        "Error code {},  message: {}".format(
            response["error_code"], response["error_message"]
        )
    )


def coval_check_retry(response):
    """Function for checking whether API returns 5xx code and if so return retry flag, so request will be made again

    Params
    ----------
    response: json
        API response in json format

    Returns
    ----------
    retry_flag: bool
        True when error code is 5xx, so request will be made again

    """
    retry_flag = False
    if response["error"]:
        logger.error(
            "Error code {},  message: {}".format(
                response["error_code"], response["error_message"]
            )
        )
        if 500 <= response["error_code"] < 600:
            retry_flag = True
    return retry_flag

In [ ]:
def get_c_chain_token_balances(addresses):
    """Function for getting info on wallets' C-Chain token balances from Covalent API

     Params
     ----------
    addresses: list
        list with C-chain addresses

     Returns
     ----------
     res: list
         list with C-chain balances information gathered from Covalent  API

    """
    res = []
    if not isinstance(addresses, list):
        logger.error("Invalid input type. List is required. Got {}".format(addresses))
        return res
    if addresses == []:
        logger.error("The input list is empty. Returning epmty list")
        return res
    for a in addresses:
        if not isinstance(a, str):
            logger.error("Invalid input type. String is required for C-chain address. Got {}".format(a))
            continue
        response = requests.get(
            covalent_endpoint
            + chain_id
            + "address/"
            + a
            + "/balances_v2/"
            + covalent_api_key
            + "&page-size=100000"
        ).json()
        if coval_check_retry(response):
            while coval_check_retry(response):
                time.sleep(1)
                response = requests.get(
                    covalent_endpoint
                    + chain_id
                    + "address/"
                    + a
                    + "/balances_v2/"
                    + covalent_api_key
                    + "&page-size=100000"
                ).json()
        try:
            response_data = {}
            response_data["address"] = a
            response_data["balances"] = response["data"]["items"]
            res.append(response_data)
        except KeyError:
            coval_response_error_logger(response)
        except TypeError:
            coval_response_error_logger(response)
        except JSONDecodeError: # I have encountered this once. Ran function again and it worked
            logger.error(
                "API Failed to respond. Please try again"
            )  
    return res

In [ ]:
def output_c_token_balances(balances):
    """
     Function for generating DataFrame out of C-chain balances info

     Params
     ----------
    balances: list
        C-chain token balances information

     Returns
     ----------
     df: pd.DataFrame
         Same information in DataFrame format
    """
    if not isinstance(balances, list):
        logger.error("Invalid input type. List is required. Got {}".format(balances))
        return None
    elif balances == []:
        logger.error("Empty list is provided for df formation. Returning None")
        return None
    elif not isinstance(balances[0], dict):
        logger.error(
            "Incorrect data format provided with token balances. Got {}".format(balances[0])
        )
        return None
    d_df = {
        "address": [],
        "balance": [],
        "contract": [],
        "contract_address": [],
        "type": [],
        "balance": [],
        "quote": [],
        "quote_rate": [],
    }
    for balance in balances:
        add_info = balance["balances"]
        add = balance["address"]
        for token_balance in add_info:
            d_df["address"].append(add)
            d_df["contract"].append(
                token_balance["contract_ticker_symbol"]
                + ": "
                + token_balance["contract_name"]
            )
            d_df["contract_address"].append(token_balance["contract_address"])
            d_df["type"].append(token_balance["type"])
            d_df["balance"].append(
                int(token_balance["balance"])
                / 10 ** int(token_balance["contract_decimals"])
            )
            d_df["quote"].append(token_balance["quote"])
            d_df["quote_rate"].append(token_balance["quote_rate"])

    df = pd.DataFrame.from_dict(d_df, orient="columns")
    return df

### Insert C-chain addresses in addresses list to get C-chain token balances

In [ ]:
addresses = [
    "0x1d7dCCBf623584009BDE3229724D29Ae5AC09d1f",
    "0x3808FD461D246D6F7BD0a02fb6bC3a77AD8922Dd",
    "0x2B1855f3E0d453faF76c91EA8504239D3bbEDb61",
    "0x0bE48D32539966c243C969ca566b7b750CF4C30b",
    "0x66f59CD59DaE5cf5d42832af087d89eC7Aaf4D74"
]
token_balances = get_c_chain_token_balances(addresses)
output_c_token_balances(token_balances)

### Function for making a request to get tx history

In [ ]:
def get_tx_data_by_address(address):
    """Function for getting info on wallets' C-Chain transaction history from Covalent API

     Params
     ----------
    address: str
        C-chain address

     Returns
     ----------
     res: list
         list with C-chain tx history information gathered from Covalent  API

    """
    res = []
    if not isinstance(address, str):
        logger.error("Invalid input type. String is required")
        return res
    response = requests.get(
        covalent_endpoint
        + chain_id
        + "address/"
        + address
        + "/transactions_v2/"
        + covalent_api_key
        + "&page-size=100000"
    ).json()
    if coval_check_retry(response):
        while coval_check_retry(response):
            time.sleep(1)
            response = requests.get(
                covalent_endpoint
                + chain_id
                + "address/"
                + address
                + "/transactions_v2/"
                + covalent_api_key
                + "&page-size=100000"
            ).json()
    try:
        res = response["data"]["items"]
    except KeyError:
        coval_response_error_logger(response)
    except TypeError:
        coval_response_error_logger(response)
    except jsonDecodeError:
        logger.error(
            "API Failed to respond. Please try again"
        )  # I have encountered this once. Ran function again and it worked
    return res

### Large function for parsing C-chain tx history based on address retrieved from Covalent API

In [ ]:
# I have based the whole swap information extraction  on transfers/withdrawals that happen along with swap event
# It is a bit complicated but this is the approach I have chosen after dilligently exploring large Tx history JSONs
# I could not get addresses from Swap event directly (I assume that some DEXes actually put both wallets names in and out info of swap event),
# because in Trader Joe swap event info both addresses were mine (wallet) addresses.
# Moreover, it would be more challenging to get token decimals information
# The downside of this approach as it does not consider interim swaps if there are more than 1 actual swap per one swap operation
# It can be done, but I assume it's out of scope of this excercise


def parse_address_txs(curr_address, txs):
    """
    Function for parsing response from Covalent API on C-chain transactions info related to a single address

    Params
    ----------
    curr_address: str
       C-chain address
    txs: dict
       C-chain transactions information

    Returns
    ----------
    transactions: dict
        Parsed information on swaps and transfers
    """
    if not isinstance(curr_address, str):
        logger.error("Invalid input type. String is required for curr_address")
        return {}
    if not isinstance(txs, list):
        logger.error("Invalid input type. List is required for txs")
        return {}
    curr_address = curr_address.lower()
    swaps_keys = [
        "type",
        "block_time",
        "block_id",
        "in_token",
        "in_value",
        "out_token",
        "out_value",
        "tx_fee",
    ]
    transfers_keys = [
        "type",
        "block_time",
        "block_id",
        "from",
        "to",
        "value",
        "value_quote",
        "tx_fee",
    ]
    transactions = {}
    for tx in txs:
        try:
            h = tx["tx_hash"]
            blh = tx["block_height"]
            s = tx["successful"]
        except KeyError:  # Here I abort the function if at least one tx has no essential fields or it is not subscriptable
            logger.error(
                tx
            )  # Since output from API follows the pattern, lack of these fields means that
            logger.error(  # input of the function (original API output) is not correct or broken
                "Problem with input transaction: KeyError"
            )
            return {}
        except TypeError:
            logger.error(tx)
            logger.error("Problem with input transaction: TypeError")
            return {}
        if tx["successful"]:
            if tx["log_events"] != []:
                inter_tx = tx["log_events"]
                for txi in inter_tx:
                    if not txi["decoded"]:
                        continue
                    if txi["decoded"]["name"] == "Swap":
                        swap = dict.fromkeys(swaps_keys)  # Initiate swap element
                        transactions[tx["tx_hash"]] = swap
                        swap["type"] = "swap"
                        swap["block_time"] = tx["block_signed_at"]
                        swap["block_id"] = tx["block_height"]
                        swap["tx_fee"] = tx["gas_spent"] * tx["gas_price"] / avax_decim
                        for ind, txi in enumerate(inter_tx):
                            if not txi["decoded"]:
                                continue
                            if (
                                txi["tx_hash"] in transactions.keys()
                                and swap["out_token"] is None
                            ):
                                if (
                                    txi["decoded"]["name"] == "Transfer"
                                    and txi["decoded"]["params"][1]["value"]
                                    == curr_address
                                ):
                                    swap["out_token"] = txi["sender_name"]
                                    swap["out_value"] = (
                                        int(txi["decoded"]["params"][-1]["value"])
                                        / 10 ** txi["sender_contract_decimals"]
                                    )
                                    break
                                elif txi["decoded"]["name"] == "Withdrawal":
                                    swap["out_token"] = txi["sender_name"]

                                    swap["out_value"] = (
                                        int(
                                            tx["log_events"][ind]["decoded"]["params"][
                                                1
                                            ]["value"]
                                        )
                                        / 10
                                        ** tx["log_events"][ind][
                                            "sender_contract_decimals"
                                        ]
                                    )
                            else:
                                continue

                            i = (
                                len(inter_tx) - 1
                            )  # backward parsing for finding first transfer from add to contract
                            while i > 0:
                                if not inter_tx[i]["decoded"]:
                                    i -= 1
                                    continue
                                if swap["in_token"] is not None:
                                    break
                                if (
                                    inter_tx[i]["decoded"]["name"] == "Transfer"
                                    and inter_tx[i]["decoded"]["params"][0]["value"]
                                    == curr_address
                                ):
                                    swap["in_token"] = inter_tx[i]["sender_name"]
                                    swap["in_value"] = (
                                        int(
                                            inter_tx[i]["decoded"]["params"][-1][
                                                "value"
                                            ]
                                        )
                                        / 10 ** inter_tx[i]["sender_contract_decimals"]
                                    )

                                elif (
                                    inter_tx[i]["decoded"]["name"] == "Deposit"
                                ):  # Pangolin works with Deposits
                                    swap["in_token"] = inter_tx[i]["sender_name"]
                                    swap["in_value"] = (
                                        int(
                                            inter_tx[i]["decoded"]["params"][1]["value"]
                                        )
                                        / 10 ** inter_tx[i]["sender_contract_decimals"]
                                    )

                                i -= 1  # ?
                    else:
                        continue
            else:
                transfer = dict.fromkeys(transfers_keys)
                transactions[tx["tx_hash"]] = transfer
                transfer["block_time"] = tx["block_signed_at"]
                transfer["block_id"] = tx["block_height"]
                transfer["tx_fee"] = tx["gas_spent"] * tx["gas_price"] / avax_decim
                transfer["from"] = tx["from_address"]
                transfer["to"] = tx["to_address"]
                transfer["value_quote"] = tx["value_quote"]
                transfer["value"] = int(tx["value"]) / avax_decim
                if tx["from_address"] == curr_address:
                    transfer["type"] = "transfer out"
                elif tx["to_address"] == curr_address:
                    transfer["type"] = "transfer in"
                else:
                    print("Error")
                info = tx["block_signed_at"], "Transfer In", int(tx["value"]) / 10 ** 18
    return transactions

### Functions for outputting dfs based on parsed data

In [ ]:
def form_txs_df(transactions):
    """
     Function for generating DataFrame out of C-chain transactions info

     Params
     ----------
    transactions: dict
        C-chain transactions information

     Returns
     ----------
     df: pd.DataFrame
         Same information in DataFrame format
    """
    keys = ["txid"]
    if not isinstance(transactions, dict):
        logger.error("Invalid input type. Dict is required for transactions")
        return None
    if transactions == {}:
        logger.error("Parsed txs info is empty")
        return None
    else:
        try:
            for (
                k,
                v,
            ) in (
                transactions.items()
            ):  
                for kk in transactions[k].keys(): # forming dict for df with all keys present in tx dict and empty lists
                    if kk not in keys:
                        keys.append(kk)
        except AttributeError:
            logger.error("Parsed txs info is incorrect, info is not dict")
            return None

        tx_dict = {k: [] for k in keys}
        for txid, info in transactions.items():  # populating lists with values
            try:
                h = info["type"]
                blh = info["block_time"]
                s = info["block_id"]
            except KeyError:
                logger.error(tx)
                logger.error("Problem with input transaction: KeyError")
                return None
            except TypeError:
                logger.error(tx)
                logger.error("Problem with input transaction: TypeError")
                return None
            tx_dict["txid"].append(txid)
            tx_dict["block_time"].append(info["block_time"])
            tx_dict["block_id"].append(info["block_id"])
            tx_dict["type"].append(info["type"])
            tx_dict["tx_fee"].append(info["tx_fee"])
            if info["type"] == "swap":
                tx_dict["in_token"].append(info["in_token"])
                tx_dict["in_value"].append(info["in_value"])
                tx_dict["out_token"].append(info["out_token"])
                tx_dict["out_value"].append(info["out_value"])
                tx_dict["from"].append(None)
                tx_dict["to"].append(None)
                tx_dict["value_quote"].append(None)
                tx_dict["value"].append(None)
            elif info["type"].startswith("transfer"):
                tx_dict["in_token"].append(None)
                tx_dict["in_value"].append(None)
                tx_dict["out_token"].append(None)
                tx_dict["out_value"].append(None)
                tx_dict["from"].append(info["from"])
                tx_dict["to"].append(info["to"])
                tx_dict["value_quote"].append(info["value_quote"])
                tx_dict["value"].append(info["value"])
                pass
            else:
                logger.error("Error while parsing transaction: ", (txid, info))
            df = pd.DataFrame.from_dict(tx_dict, orient="columns")
        return df


def output_txs_df(transactions_df):
    """
     Function for outputting general txs info out of C-chain transactions dataframe

     Params
     ----------
    transactions_df: dict
        C-chain transactions information

     Returns
     ----------
     txs_info: pd.DataFrame
         Filtered general information about txs
    """
    try:
        txs_info = transactions_df[["txid", "type", "block_id", "block_time", "tx_fee"]]
    except Exception as e:
        logger.error("Issue with parsing transactions df: ", e)
    return txs_info


def output_transfers_df(transactions_df):
    """
     Function for outputting detailed transfers info out of C-chain transactions dataframe

     Params
     ----------
    transactions_df: pd.DataFrame
        C-chain transactions information

     Returns
     ----------
     transf_info: pd.DataFrame
         Filtered information about transfers
    """
    try:
        transf_info = transactions_df[transactions_df.type.str.startswith("transfer")]
        transf_info = transf_info[
            [
                "txid",
                "type",
                "block_id",
                "block_time",
                "from",
                "to",
                "value",
                "value_quote",
                "tx_fee",
            ]
        ]
    except Exception as e:
        logger.error(" Issue with parsing transactions df: ", e)
    return transf_info


def output_swaps_df(transactions_df):
    """
     Function for outputting detailed swaps info out of C-chain transactions dataframe

     Params
     ----------
    transactions_df: pd.DataFrame
        C-chain transactions information

     Returns
     ----------
     swaps_info: pd.DataFrame
         Filtered information about swaps
    """
    swaps_info = transactions_df[transactions_df.type.str.startswith("swap")]
    swaps_info = swaps_info[
        [
            "txid",
            "block_id",
            "block_time",
            "in_token",
            "in_value",
            "out_token",
            "out_value",
            "tx_fee",
        ]
    ]
    return swaps_info

### Insert C-chain address in curr_address to get tx history and output general tx data
#### This section might be running slow in case there is a large number of txs in address history

In [ ]:
curr_address = "0x300dd6dfc039e7842a5d897bb67dfc2c3ee687d8"  # Input address here
txs = get_tx_data_by_address(curr_address)
parsed_txs = parse_address_txs(curr_address, txs)
txs_dataframe = form_txs_df(parsed_txs)
output_txs_df(txs_dataframe)

### Output detailed data on transfers

In [ ]:
output_transfers_df(txs_dataframe)

### Output detailed data on swaps

In [ ]:
output_swaps_df(txs_dataframe)

## Testing

### Testing constants

In [ ]:
valid_x_add = "X-avax17acd63tqk6f9sv4482n80lpv4qaxv53xcv5y4v"
valid_c_add = "0x300DD6dfc039e7842A5D897bB67dFC2c3ee687d8"
avax_native_asset = {
    "assetID": "2UBD6evUf2qo1JxjnadAtcrGj4Eo1Kbeu4YCTB7JefnR8krtMP",
    "name": "PartySwap Silver Entry V2",
    "symbol": "VPSX",
    "denomination": "0",
}

In [ ]:
valid_token_balances = [
    {
        "address": "0x1d7dCCBf623584009BDE3229724D29Ae5AC09d1f",
        "balances": [
            {
                "contract_decimals": 18,
                "contract_name": "Avalanche Coin",
                "contract_ticker_symbol": "AVAX",
                "contract_address": "0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee",
                "supports_erc": None,
                "logo_url": "https://www.covalenthq.com/static/images/icons/display-icons/avalanche-avax-logo.png",
                "last_transferred_at": None,
                "type": "cryptocurrency",
                "balance": "0",
                "balance_24h": None,
                "quote_rate": 83.0323,
                "quote_rate_24h": 84.67252,
                "quote": 0.0,
                "quote_24h": None,
                "nft_data": None,
            }
        ],
    },
    {
        "address": "0x3808FD461D246D6F7BD0a02fb6bC3a77AD8922Dd",
        "balances": [
            {
                "contract_decimals": 18,
                "contract_name": "Avalanche Coin",
                "contract_ticker_symbol": "AVAX",
                "contract_address": "0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee",
                "supports_erc": None,
                "logo_url": "https://www.covalenthq.com/static/images/icons/display-icons/avalanche-avax-logo.png",
                "last_transferred_at": None,
                "type": "cryptocurrency",
                "balance": "0",
                "balance_24h": None,
                "quote_rate": 83.1673,
                "quote_rate_24h": 84.67252,
                "quote": 0.0,
                "quote_24h": None,
                "nft_data": None,
            }
        ],
    },
    {
        "address": "0x2B1855f3E0d453faF76c91EA8504239D3bbEDb61",
        "balances": [
            {
                "contract_decimals": 18,
                "contract_name": "Avalanche Coin",
                "contract_ticker_symbol": "AVAX",
                "contract_address": "0xeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeeee",
                "supports_erc": None,
                "logo_url": "https://www.covalenthq.com/static/images/icons/display-icons/avalanche-avax-logo.png",
                "last_transferred_at": None,
                "type": "cryptocurrency",
                "balance": "605332239489601860",
                "balance_24h": None,
                "quote_rate": 83.0323,
                "quote_rate_24h": 84.67252,
                "quote": 50.26213,
                "quote_24h": None,
                "nft_data": None,
            },
            {
                "contract_decimals": 6,
                "contract_name": "DoloElk",
                "contract_ticker_symbol": "DOLO",
                "contract_address": "0xb1b9e9274f864862576f80b0f7a2df25e608b070",
                "supports_erc": ["erc20"],
                "logo_url": "https://logos.covalenthq.com/tokens/43114/0xb1b9e9274f864862576f80b0f7a2df25e608b070.png",
                "last_transferred_at": "2022-02-18T00:15:48Z",
                "type": "cryptocurrency",
                "balance": "130488342",
                "balance_24h": None,
                "quote_rate": 0.011642184,
                "quote_rate_24h": 0.0118541205,
                "quote": 1.5191693,
                "quote_24h": None,
                "nft_data": None,
            },
            {
                "contract_decimals": 18,
                "contract_name": "AVAX MILK",
                "contract_ticker_symbol": "avaMILK",
                "contract_address": "0x6243527cb09fd4e4e398181f6f3e09285926567e",
                "supports_erc": ["erc20"],
                "logo_url": "https://logos.covalenthq.com/tokens/43114/0x6243527cb09fd4e4e398181f6f3e09285926567e.png",
                "last_transferred_at": "2022-03-01T16:25:52Z",
                "type": "cryptocurrency",
                "balance": "39767416084301119797921",
                "balance_24h": None,
                "quote_rate": 5.05463e-06,
                "quote_rate_24h": 5.1466454e-06,
                "quote": 0.20100957,
                "quote_24h": None,
                "nft_data": None,
            },
            {
                "contract_decimals": 6,
                "contract_name": "USD Coin",
                "contract_ticker_symbol": "USDC.e",
                "contract_address": "0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664",
                "supports_erc": ["erc20"],
                "logo_url": "https://logos.covalenthq.com/tokens/43114/0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664.png",
                "last_transferred_at": "2022-02-22T17:25:11Z",
                "type": "dust",
                "balance": "0",
                "balance_24h": None,
                "quote_rate": 0.9986553,
                "quote_rate_24h": 0.9725717,
                "quote": 0.0,
                "quote_24h": None,
                "nft_data": None,
            },
            {
                "contract_decimals": 18,
                "contract_name": "Wrapped AVAX",
                "contract_ticker_symbol": "WAVAX",
                "contract_address": "0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7",
                "supports_erc": ["erc20"],
                "logo_url": "https://logos.covalenthq.com/tokens/43114/0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7.png",
                "last_transferred_at": "2022-02-18T00:15:48Z",
                "type": "dust",
                "balance": "0",
                "balance_24h": None,
                "quote_rate": 83.0214,
                "quote_rate_24h": 84.46461,
                "quote": 0.0,
                "quote_24h": None,
                "nft_data": None,
            },
            {
                "contract_decimals": 6,
                "contract_name": "Tether USD",
                "contract_ticker_symbol": "USDT.e",
                "contract_address": "0xc7198437980c041c805a1edcba50c1ce5db95118",
                "supports_erc": ["erc20"],
                "logo_url": "https://logos.covalenthq.com/tokens/43114/0xc7198437980c041c805a1edcba50c1ce5db95118.png",
                "last_transferred_at": "2022-02-19T00:17:40Z",
                "type": "dust",
                "balance": "0",
                "balance_24h": None,
                "quote_rate": 0.99909145,
                "quote_rate_24h": 0.97559744,
                "quote": 0.0,
                "quote_24h": None,
                "nft_data": None,
            },
            {
                "contract_decimals": 18,
                "contract_name": "Elk",
                "contract_ticker_symbol": "ELK",
                "contract_address": "0xe1c110e1b1b4a1ded0caf3e42bfbdbb7b5d7ce1c",
                "supports_erc": ["erc20"],
                "logo_url": "https://logos.covalenthq.com/tokens/43114/0xe1c110e1b1b4a1ded0caf3e42bfbdbb7b5d7ce1c.png",
                "last_transferred_at": "2022-02-18T16:33:34Z",
                "type": "dust",
                "balance": "856009458263808",
                "balance_24h": None,
                "quote_rate": 2.3493514,
                "quote_rate_24h": 2.3384945,
                "quote": 0.0,
                "quote_24h": None,
                "nft_data": None,
            },
        ],
    },
]
valid_parsed_txs = {
    "0x7499222dcc556e7024066ea3e27fc1ae8acf01ce9b317fc05a9c28485e2f165f": {
        "type": "transfer out",
        "block_time": "2021-11-11T19:36:05Z",
        "block_id": 6820871,
        "from": "0x300dd6dfc039e7842a5d897bb67dfc2c3ee687d8",
        "to": "0x2ebfc9b4f9d48bf2647f9b5f5224f6b0385458e1",
        "value": 12.315233931279753,
        "value_quote": 1084.8800119313523,
        "tx_fee": 0.000525,
    },
    "0x173a0325ad159c0f55db9085339a4ad03d7a7257f24788660e3fff8243e22383": {
        "type": "swap",
        "block_time": "2021-11-11T19:34:52Z",
        "block_id": 6820835,
        "in_token": "USD Coin",
        "in_value": 1082.133691,
        "out_token": "Wrapped AVAX",
        "out_value": 12.287614897395931,
        "tx_fee": 0.00357855,
    },
    "0x4391fde346323b8d2f7adf0061d22a54b307591d1836e1fc4be3ecb45b8e54ea": {
        "type": "swap",
        "block_time": "2021-11-10T21:22:34Z",
        "block_id": 6781574,
        "in_token": "Husky",
        "in_value": 8487315679.99636,
        "out_token": "USD Coin",
        "out_value": 1082.133691,
        "tx_fee": 0.010638991765859508,
    },
}
valid_balances_result_1st = {
    "address": "0x300DD6dfc039e7842A5D897bB67dFC2c3ee687d8",
    "balances": {
        "contract_decimals": 18,
        "contract_name": "Husky",
        "contract_ticker_symbol": "HUSKY",
        "contract_address": "0x65378b697853568da9ff8eab60c13e1ee9f4a654",
        "supports_erc": ["erc20"],
        "logo_url": "https://logos.covalenthq.com/tokens/43114/0x65378b697853568da9ff8eab60c13e1ee9f4a654.png",
        "last_transferred_at": "2021-11-10T21:22:34Z",
        "type": "dust",
        "balance": "0",
        "balance_24h": None,
        "quote_rate": 4.399262e-08,
        "quote_rate_24h": 4.4156497e-08,
        "quote": 0.0,
        "quote_24h": None,
        "nft_data": None,
    },
}
valid_txs = [
    {
        "block_signed_at": "2021-11-11T19:36:05Z",
        "block_height": 6820871,
        "tx_hash": "0x7499222dcc556e7024066ea3e27fc1ae8acf01ce9b317fc05a9c28485e2f165f",
        "tx_offset": 12,
        "successful": True,
        "from_address": "0x300dd6dfc039e7842a5d897bb67dfc2c3ee687d8",
        "from_address_label": None,
        "to_address": "0x2ebfc9b4f9d48bf2647f9b5f5224f6b0385458e1",
        "to_address_label": None,
        "value": "12315233931279752298",
        "value_quote": 1084.8800119313523,
        "gas_offered": 21000,
        "gas_spent": 21000,
        "gas_price": 25000000000,
        "gas_quote": 0.046248573875427246,
        "gas_quote_rate": 88.09252166748047,
        "log_events": [],
    },
    {
        "block_signed_at": "2021-11-11T19:34:52Z",
        "block_height": 6820835,
        "tx_hash": "0x173a0325ad159c0f55db9085339a4ad03d7a7257f24788660e3fff8243e22383",
        "tx_offset": 9,
        "successful": True,
        "from_address": "0x300dd6dfc039e7842a5d897bb67dfc2c3ee687d8",
        "from_address_label": None,
        "to_address": "0x60ae616a2155ee3d9a68541ba4544862310933d4",
        "to_address_label": None,
        "value": "0",
        "value_quote": 0.0,
        "gas_offered": 174028,
        "gas_spent": 143142,
        "gas_price": 25000000000,
        "gas_quote": 0.3152434934131622,
        "gas_quote_rate": 88.09252166748047,
        "log_events": [
            {
                "block_signed_at": "2021-11-11T19:34:52Z",
                "block_height": 6820835,
                "tx_offset": 9,
                "log_offset": 57,
                "tx_hash": "0x173a0325ad159c0f55db9085339a4ad03d7a7257f24788660e3fff8243e22383",
                "raw_log_topics": [
                    "0x7fcf532c15f0a6db0bd6d0e038bea71d30d808c7d98cb3bf7268a95bf5081b65",
                    "0x00000000000000000000000060ae616a2155ee3d9a68541ba4544862310933d4",
                ],
                "sender_contract_decimals": 18,
                "sender_name": "Wrapped AVAX",
                "sender_contract_ticker_symbol": "WAVAX",
                "sender_address": "0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7",
                "sender_address_label": None,
                "sender_logo_url": "https://logos.covalenthq.com/tokens/0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7.png",
                "raw_log_data": "0x000000000000000000000000000000000000000000000000aa8660a588151a8f",
                "decoded": {
                    "name": "Withdrawal",
                    "signature": "Withdrawal(indexed address src, uint256 wad)",
                    "params": [
                        {
                            "name": "src",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0x60ae616a2155ee3d9a68541ba4544862310933d4",
                        },
                        {
                            "name": "wad",
                            "type": "uint256",
                            "indexed": False,
                            "decoded": True,
                            "value": "12287614897395931791",
                        },
                    ],
                },
            },
            {
                "block_signed_at": "2021-11-11T19:34:52Z",
                "block_height": 6820835,
                "tx_offset": 9,
                "log_offset": 56,
                "tx_hash": "0x173a0325ad159c0f55db9085339a4ad03d7a7257f24788660e3fff8243e22383",
                "raw_log_topics": [
                    "0xd78ad95fa46c994b6551d0da85fc275fe613ce37657fb8d5e3d130840159d822",
                    "0x00000000000000000000000060ae616a2155ee3d9a68541ba4544862310933d4",
                    "0x00000000000000000000000060ae616a2155ee3d9a68541ba4544862310933d4",
                ],
                "sender_contract_decimals": 18,
                "sender_name": "Joe LP Token",
                "sender_contract_ticker_symbol": "JLP",
                "sender_address": "0xa389f9430876455c36478deea9769b7ca4e3ddb1",
                "sender_address_label": None,
                "sender_logo_url": "https://logos.covalenthq.com/tokens/43114/0xa389f9430876455c36478deea9769b7ca4e3ddb1.png",
                "raw_log_data": "0x0000000000000000000000000000000000000000000000000000000040800cbb00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000aa8660a588151a8f",
                "decoded": {
                    "name": "Swap",
                    "signature": "Swap(indexed address sender, uint256 amount0In, uint256 amount1In, uint256 amount0Out, uint256 amount1Out, indexed address to)",
                    "params": [
                        {
                            "name": "sender",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0x60ae616a2155ee3d9a68541ba4544862310933d4",
                        },
                        {
                            "name": "amount0In",
                            "type": "uint256",
                            "indexed": False,
                            "decoded": True,
                            "value": "1082133691",
                        },
                        {
                            "name": "amount1In",
                            "type": "uint256",
                            "indexed": False,
                            "decoded": True,
                            "value": "0",
                        },
                        {
                            "name": "amount0Out",
                            "type": "uint256",
                            "indexed": False,
                            "decoded": True,
                            "value": "0",
                        },
                        {
                            "name": "amount1Out",
                            "type": "uint256",
                            "indexed": False,
                            "decoded": True,
                            "value": "12287614897395931791",
                        },
                        {
                            "name": "to",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0x60ae616a2155ee3d9a68541ba4544862310933d4",
                        },
                    ],
                },
            },
            {
                "block_signed_at": "2021-11-11T19:34:52Z",
                "block_height": 6820835,
                "tx_offset": 9,
                "log_offset": 55,
                "tx_hash": "0x173a0325ad159c0f55db9085339a4ad03d7a7257f24788660e3fff8243e22383",
                "raw_log_topics": [
                    "0x1c411e9a96e071241c2f21f7726b17ae89e3cab4c78be50e062b03a9fffbbad1"
                ],
                "sender_contract_decimals": 18,
                "sender_name": "Joe LP Token",
                "sender_contract_ticker_symbol": "JLP",
                "sender_address": "0xa389f9430876455c36478deea9769b7ca4e3ddb1",
                "sender_address_label": None,
                "sender_logo_url": "https://logos.covalenthq.com/tokens/43114/0xa389f9430876455c36478deea9769b7ca4e3ddb1.png",
                "raw_log_data": "0x00000000000000000000000000000000000000000000000000004bcb20c6218500000000000000000000000000000000000000000000c8fb7798b2fcac16582c",
                "decoded": {
                    "name": "Sync",
                    "signature": "Sync(uint112 reserve0, uint112 reserve1)",
                    "params": [
                        {
                            "name": "reserve0",
                            "type": "uint112",
                            "indexed": False,
                            "decoded": True,
                            "value": "83335800299909",
                        },
                        {
                            "name": "reserve1",
                            "type": "uint112",
                            "indexed": False,
                            "decoded": True,
                            "value": "949112047171115402287148",
                        },
                    ],
                },
            },
            {
                "block_signed_at": "2021-11-11T19:34:52Z",
                "block_height": 6820835,
                "tx_offset": 9,
                "log_offset": 54,
                "tx_hash": "0x173a0325ad159c0f55db9085339a4ad03d7a7257f24788660e3fff8243e22383",
                "raw_log_topics": [
                    "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef",
                    "0x000000000000000000000000a389f9430876455c36478deea9769b7ca4e3ddb1",
                    "0x00000000000000000000000060ae616a2155ee3d9a68541ba4544862310933d4",
                ],
                "sender_contract_decimals": 18,
                "sender_name": "Wrapped AVAX",
                "sender_contract_ticker_symbol": "WAVAX",
                "sender_address": "0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7",
                "sender_address_label": None,
                "sender_logo_url": "https://logos.covalenthq.com/tokens/0xb31f66aa3c1e785363f0875a1b74e27b85fd66c7.png",
                "raw_log_data": "0x000000000000000000000000000000000000000000000000aa8660a588151a8f",
                "decoded": {
                    "name": "Transfer",
                    "signature": "Transfer(indexed address from, indexed address to, uint256 value)",
                    "params": [
                        {
                            "name": "from",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0xa389f9430876455c36478deea9769b7ca4e3ddb1",
                        },
                        {
                            "name": "to",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0x60ae616a2155ee3d9a68541ba4544862310933d4",
                        },
                        {
                            "name": "value",
                            "type": "uint256",
                            "indexed": False,
                            "decoded": True,
                            "value": "12287614897395931791",
                        },
                    ],
                },
            },
            {
                "block_signed_at": "2021-11-11T19:34:52Z",
                "block_height": 6820835,
                "tx_offset": 9,
                "log_offset": 53,
                "tx_hash": "0x173a0325ad159c0f55db9085339a4ad03d7a7257f24788660e3fff8243e22383",
                "raw_log_topics": [
                    "0x8c5be1e5ebec7d5bd14f71427d1e84f3dd0314c0f7b2291e5b200ac8c7c3b925",
                    "0x000000000000000000000000300dd6dfc039e7842a5d897bb67dfc2c3ee687d8",
                    "0x00000000000000000000000060ae616a2155ee3d9a68541ba4544862310933d4",
                ],
                "sender_contract_decimals": 6,
                "sender_name": "USD Coin",
                "sender_contract_ticker_symbol": "USDC.e",
                "sender_address": "0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664",
                "sender_address_label": None,
                "sender_logo_url": "https://logos.covalenthq.com/tokens/0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664.png",
                "raw_log_data": "0xffffffffffffffffffffffffffffffffffffffffffffffffffffffffbf7ff344",
                "decoded": {
                    "name": "Approval",
                    "signature": "Approval(indexed address owner, indexed address spender, uint256 value)",
                    "params": [
                        {
                            "name": "owner",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0x300dd6dfc039e7842a5d897bb67dfc2c3ee687d8",
                        },
                        {
                            "name": "spender",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0x60ae616a2155ee3d9a68541ba4544862310933d4",
                        },
                        {
                            "name": "value",
                            "type": "uint256",
                            "indexed": False,
                            "decoded": True,
                            "value": "115792089237316195423570985008687907853269984665640564039457584007912047506244",
                        },
                    ],
                },
            },
            {
                "block_signed_at": "2021-11-11T19:34:52Z",
                "block_height": 6820835,
                "tx_offset": 9,
                "log_offset": 52,
                "tx_hash": "0x173a0325ad159c0f55db9085339a4ad03d7a7257f24788660e3fff8243e22383",
                "raw_log_topics": [
                    "0xddf252ad1be2c89b69c2b068fc378daa952ba7f163c4a11628f55a4df523b3ef",
                    "0x000000000000000000000000300dd6dfc039e7842a5d897bb67dfc2c3ee687d8",
                    "0x000000000000000000000000a389f9430876455c36478deea9769b7ca4e3ddb1",
                ],
                "sender_contract_decimals": 6,
                "sender_name": "USD Coin",
                "sender_contract_ticker_symbol": "USDC.e",
                "sender_address": "0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664",
                "sender_address_label": None,
                "sender_logo_url": "https://logos.covalenthq.com/tokens/0xa7d7079b0fead91f3e65f86e8915cb59c1a4c664.png",
                "raw_log_data": "0x0000000000000000000000000000000000000000000000000000000040800cbb",
                "decoded": {
                    "name": "Transfer",
                    "signature": "Transfer(indexed address from, indexed address to, uint256 value)",
                    "params": [
                        {
                            "name": "from",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0x300dd6dfc039e7842a5d897bb67dfc2c3ee687d8",
                        },
                        {
                            "name": "to",
                            "type": "address",
                            "indexed": True,
                            "decoded": True,
                            "value": "0xa389f9430876455c36478deea9769b7ca4e3ddb1",
                        },
                        {
                            "name": "value",
                            "type": "uint256",
                            "indexed": False,
                            "decoded": True,
                            "value": "1082133691",
                        },
                    ],
                },
            },
        ],
    },
]

### Test cases

In [ ]:
class TestGet_avalanche_c_balance(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertDictEqual(get_avalanche_c_balance([]), {})
        self.assertDictEqual(get_avalanche_c_balance(["teststring"]), {})
        self.assertDictEqual(get_avalanche_c_balance("teststring"), {})
        self.assertDictEqual(get_avalanche_c_balance(12), {})
        self.assertDictEqual(get_avalanche_c_balance(valid_c_add), {})
        self.assertDictEqual(
            get_avalanche_c_balance([valid_x_add]), {}
        )  # I enter only 1 address because I am sure about its balance

    def test_valid_input(self):
        self.assertDictEqual(get_avalanche_c_balance([valid_c_add]), {valid_c_add: 0.0})


class TestGet_avalanche_x_balances(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertDictEqual(get_avalanche_x_balances([]), {})
        self.assertDictEqual(get_avalanche_x_balances(["teststring"]), {})
        self.assertDictEqual(get_avalanche_x_balances("teststring"), {})
        self.assertDictEqual(get_avalanche_x_balances(12), {})
        self.assertDictEqual(get_avalanche_x_balances([valid_c_add]), {})
        self.assertDictEqual(
            get_avalanche_x_balances(["avax17acd63tqk6f9sv4482n80lpv4qaxv53xcv5y4v"]),
            {},
        )  # no separator

    def test_valid_input(self):
        self.assertDictEqual(get_avalanche_x_balances([valid_x_add]), {valid_x_add: []})


class TestGet_asset_description(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertDictEqual(get_asset_description([avax_native_asset["assetID"]]), {})
        self.assertDictEqual(get_asset_description("teststring"), {})
        self.assertDictEqual(get_asset_description(12), {})
        self.assertDictEqual(get_asset_description([valid_c_add]), {})
        self.assertDictEqual(
            get_asset_description(["avax17acd63tqk6f9sv4482n80lpv4qaxv53xcv5y4v"]), {}
        )

    def test_valid_input(self):
        self.assertDictEqual(
            get_asset_description(avax_native_asset["assetID"]), avax_native_asset
        )


class TestGet_c_chain_token_balances(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertListEqual(
            get_c_chain_token_balances([avax_native_asset["assetID"]]), []
        )
        self.assertListEqual(get_c_chain_token_balances("teststring"), [])
        self.assertListEqual(get_c_chain_token_balances(12), [])
        self.assertListEqual(get_c_chain_token_balances(valid_c_add), [])
        self.assertListEqual(get_c_chain_token_balances(valid_x_add), [])

    def test_valid_input(self):
        self.assertNotEqual(get_c_chain_token_balances([valid_c_add]), [])
        self.assertIsInstance(get_c_chain_token_balances([valid_c_add]), list)

    def test_output_validity(self):
        first_tx = get_c_chain_token_balances([valid_c_add])[0]
        self.assertIsInstance(first_tx, dict)
        self.assertNotEqual(first_tx, {})
        self.assertIn("address", first_tx, {})
        self.assertIn("balances", first_tx, {})
        self.assertIsNotNone(first_tx["balances"])
        self.assertIsNotNone(first_tx["address"])


class TestOutput_c_token_balances(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertIsNone(output_c_token_balances({}))
        self.assertIsNone(output_c_token_balances("teststring"))
        self.assertIsNone(output_c_token_balances(12))
        self.assertIsNone(output_c_token_balances([valid_c_add]))
        self.assertIsNone(output_c_token_balances(valid_c_add))

    def test_valid_input(self):
        self.assertIsInstance(
            output_c_token_balances(valid_token_balances), pd.DataFrame
        )


class TestGet_tx_data_by_address(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertListEqual(get_tx_data_by_address([avax_native_asset["assetID"]]), [])
        self.assertListEqual(get_tx_data_by_address("teststring"), [])
        self.assertListEqual(get_tx_data_by_address(12), [])
        self.assertListEqual(get_tx_data_by_address([valid_c_add]), [])
        self.assertListEqual(get_tx_data_by_address(valid_x_add), [])

    def test_valid_input(self):
        self.assertNotEqual(get_tx_data_by_address(valid_c_add), [])
        self.assertIsInstance(get_tx_data_by_address(valid_c_add), list)
        self.assertIsInstance(get_tx_data_by_address(valid_c_add)[0], dict)

    def test_output_validity(self):
        first_tx = get_tx_data_by_address(valid_c_add)[0]
        self.assertNotEqual(first_tx, {})
        self.assertIn("tx_hash", first_tx)
        self.assertIn("value", first_tx)
        self.assertIn("block_height", first_tx)
        self.assertIn("block_signed_at", first_tx)


class TestOutput_x_balances(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertIsNone(output_x_balances({}))
        self.assertIsNone(output_x_balances("teststring"))
        self.assertIsNone(output_x_balances(12))
        self.assertIsNone(output_x_balances([valid_x_add]))
        self.assertIsNone(output_x_balances(valid_x_add))

    def test_valid_input(self):
        self.assertIsInstance(
            output_x_balances(
                {valid_x_add: [{"asset": "AVAX", "balance": "291078658397911"}]}
            ),
            pd.DataFrame,
        )


class TestOutput_c_balances(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertIsNone(output_c_balances({}))
        self.assertIsNone(output_c_balances("teststring"))
        self.assertIsNone(output_c_balances(12))
        self.assertIsNone(output_c_balances([valid_c_add]))
        self.assertIsNone(output_c_balances(valid_c_add))

    def test_valid_input(self):
        self.assertIsInstance(output_c_balances({valid_c_add: 100}), pd.DataFrame)


class TestParse_tx(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertDictEqual(parse_address_txs("teststring", {}), {})
        self.assertDictEqual(parse_address_txs("teststring", []), {})
        self.assertDictEqual(parse_address_txs(valid_c_add, []), {})
        self.assertDictEqual(parse_address_txs(valid_c_add, [{"tx": "tx"}]), {})
        self.assertDictEqual(
            parse_address_txs(
                valid_c_add,
                [
                    {
                        "name": "value",
                        "type": "uint256",
                        "indexed": False,
                        "decoded": True,
                        "value": "1082133691",
                    }
                ],
            ),
            {},
        )

    def test_valid_input(self):
        self.assertIsInstance(parse_address_txs(valid_c_add, valid_txs), dict)

    def test_output_validity(self):
        parsed_output = parse_address_txs(valid_c_add, valid_txs)
        self.assertIn(
            "0x7499222dcc556e7024066ea3e27fc1ae8acf01ce9b317fc05a9c28485e2f165f",
            parsed_output,
        )
        parsed_first_tx = parsed_output[
            "0x7499222dcc556e7024066ea3e27fc1ae8acf01ce9b317fc05a9c28485e2f165f"
        ]
        self.assertIn("type", parsed_first_tx)
        self.assertIn("value", parsed_first_tx)
        self.assertIn("block_id", parsed_first_tx)
        self.assertIn("tx_fee", parsed_first_tx)


class TestForm_txs_df(unittest.TestCase):
    def test_invalid_inputs(self):
        self.assertIsNone(form_txs_df({}))
        self.assertIsNone(form_txs_df({"test_field": 12}))
        self.assertIsNone(form_txs_df("transactions"))

    def test_valid_input(self):
        self.assertIsInstance(form_txs_df(valid_parsed_txs), pd.DataFrame)

### Run tests

In [ ]:
if __name__ == "__main__":
    unittest.main(argv=["first-arg-is-ignored"], exit=False)